In [1]:
import sys
sys.path.append("/home/schmid/Text2Head/NPHM")

import torch
from torch.optim import Adam
from utils.pipeline import forward, get_latent_mean_std, get_latent_from_text
import optuna

torch.cuda.empty_cache()

/home/schmid/miniconda3/envs/nphm/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


This notebook serves to find the best hyperparameters for the optimization.
Important:
- to make different runs comparable, we always start from lat_mean
- for the model to be able to yield good results for varying prompts, we use 3 prompts from different categories (gender, ethnicity, hairstyle) for the hyperparameter tuning

In [2]:
hparams = {
        'resolution': 120,
        'n_iterations': 80,
        'optimizer_lr': 1.4e-3,
        'lr_scheduler_factor': 0.53,
        'lr_scheduler_patience': 5,
        'lr_scheduler_min_lr': 2.6e-5
    }

prompts = []
prompts.append('A woman')
prompts.append('An Asian person')
prompts.append('A person with beard')

def latent_optimization(hparams):
    all_CLIP_scores = []
    for prompt in prompts:
        lat_mean, lat_std = get_latent_mean_std()
        _, best_score, _ = get_latent_from_text(prompt, hparams, init_lat=lat_mean)
        all_CLIP_scores.append(best_score)

    all_CLIP_scores_tensor = torch.stack(all_CLIP_scores)
    print(all_CLIP_scores_tensor)
    # Calculate the average
    average_score = torch.mean(all_CLIP_scores_tensor)

    return average_score

def objective(trial):

    search_space = {
        'resolution': 120,
        'n_iterations': trial.suggest_categorical('n_iterations', [50, 60, 70, 80]),
        'optimizer_lr': trial.suggest_float('optimizer_lr', 1e-4, 5e-3, log=True),
        'lr_scheduler_factor': trial.suggest_float('lr_scheduler_factor', 0.5, 0.9),
        'lr_scheduler_patience': trial.suggest_int('lr_scheduler_patience', 3, 6),
        'lr_scheduler_min_lr': trial.suggest_float('lr_scheduler_minlr', 1e-5, 1e-4, log=True),
    }

    hparams.update(search_space)

    return latent_optimization(hparams)

In [3]:
if __name__ == '__main__':
    study = optuna.create_study(storage="sqlite:///../optuna_study_hparams.db", study_name="optim_hparams_update1", direction='maximize', load_if_exists=True)
    study.optimize(objective, n_trials=2)
        
    best_params = study.best_params
    print(best_params)

[I 2023-12-09 22:43:34,115] Using an existing study with name 'optim_hparams_update1' instead of creating a new one.


  0%|          | 0/80 [00:00<?, ?it/s]/home/schmid/Text2Head/utils/pipeline.py:78: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3277.)
  prob = -delta.T @ torch.inverse(cov) @ delta
  1%|▏         | 1/80 [00:02<02:40,  2.03s/it]

update step 1 - score: tensor([[19.3760]], grad_fn=<AddBackward0>)


  2%|▎         | 2/80 [00:03<02:30,  1.93s/it]

update step 2 - score: tensor([[20.4435]], grad_fn=<AddBackward0>)


...

update step 78 - score: tensor([[28.8648]], grad_fn=<AddBackward0>)


 99%|█████████▉| 79/80 [02:31<00:01,  1.90s/it]

update step 79 - score: tensor([[28.9408]], grad_fn=<AddBackward0>)


100%|██████████| 80/80 [02:33<00:00,  1.92s/it]
[I 2023-12-09 22:59:06,119] Trial 23 finished with value: 28.48662757873535 and parameters: {'n_iterations': 80, 'optimizer_lr': 0.002185180086992407, 'lr_scheduler_factor': 0.6617620486429487, 'lr_scheduler_patience': 3, 'lr_scheduler_minlr': 4.40256149452234e-05}. Best is trial 5 with value: 28.936965942382812.


update step 80 - score: tensor([[29.1194]], grad_fn=<AddBackward0>)
tensor([[[25.9983]],

        [[30.3423]],

        [[29.1194]]])
{'n_iterations': 80, 'optimizer_lr': 0.001412201527890809, 'lr_scheduler_factor': 0.5339593253335609, 'lr_scheduler_patience': 5, 'lr_scheduler_minlr': 2.6266529797318665e-05}
